# Let's do some awesome Choropleth Maps!

In [1]:
# Install Plotly-geo
!pip install plotly-geo
!pip install geopandas==0.3.0
!pip install pyshp==1.2.10
!pip install shapely==1.6.3

     |████████████████████████████████| 23.7MB 96kB/s 
     |████████████████████████████████| 890kB 4.9MB/s 
     |████████████████████████████████| 11.8MB 200kB/s 
     |████████████████████████████████| 10.1MB 33.2MB/s 
     |████████████████████████████████| 184kB 4.9MB/s 
  Created wheel for pyshp: filename=pyshp-1.2.10-cp36-none-any.whl size=20468 sha256=3429b52852e6270bba69e35565b635f5974006c5ef65302af3de40622a4b106a
  Stored in directory: /root/.cache/pip/wheels/ba/1a/67/6a12977f362c33a15edc753daf92c6f01879dbf4db76faf0dd
Successfully built pyshp
     |████████████████████████████████| 1.5MB 5.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Shapely 1.6.4.post2
    Uninstalling Shapely-1.6.4.post2:
      Successfully uninstalled Shapely-1.6.4.post2


In [2]:
# First we impot plotly's figure factory
import plotly.figure_factory as ff

import numpy as np
import pandas as pd

# Load our data
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_sample.head()

LAUS Code  State FIPS Code  ...  Unemployed Unemployment Rate (%)
0  CN0100100000000                1  ...  1,352                        5.3
1  CN0100300000000                1  ...  4,870                        5.4
2  CN0100500000000                1  ...    718                        8.6
3  CN0100700000000                1  ...    569                        6.6
4  CN0100900000000                1  ...  1,354                        5.5

[5 rows x 9 columns]

The columns or features we're interested in using for ur plots are the State FIPS Code and County FIPS code, but we'll need to combine them to make it into a full zip code

In [3]:
# Now let's format the State FIPS Code into the format needed. i.e with a zero before the one
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample.head()

LAUS Code State FIPS Code  ...  Unemployed Unemployment Rate (%)
0  CN0100100000000              01  ...  1,352                        5.3
1  CN0100300000000              01  ...  4,870                        5.4
2  CN0100500000000              01  ...    718                        8.6
3  CN0100700000000              01  ...    569                        6.6
4  CN0100900000000              01  ...  1,354                        5.5

[5 rows x 9 columns]

In [4]:
# Creating our FIPS
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']
df_sample.head()

LAUS Code State FIPS Code  ... Unemployment Rate (%)   FIPS
0  CN0100100000000              01  ...                   5.3  01001
1  CN0100300000000              01  ...                   5.4  01003
2  CN0100500000000              01  ...                   8.6  01005
3  CN0100700000000              01  ...                   6.6  01007
4  CN0100900000000              01  ...                   5.5  01009

[5 rows x 10 columns]

In [0]:
# We define our color scale, this is simply the colors we use for each category class in unemployed
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]

# Define our categories using np.linspace
endpts = list(np.linspace(1, 12, len(colorscale) - 1))

In [9]:
# Let's view our endpts
endpts

[1.0,
 1.7333333333333334,
 2.466666666666667,
 3.1999999999999997,
 3.933333333333333,
 4.666666666666666,
 5.3999999999999995,
 6.133333333333333,
 6.866666666666666,
 7.6,
 8.333333333333332,
 9.066666666666666,
 9.799999999999999,
 10.533333333333333,
 11.266666666666666,
 12.0]

In [10]:
# How linspace works
np.linspace(1.0, 3.0, num=5)

array([1. , 1.5, 2. , 2.5, 3. ])

In [0]:
# put our FIPS into a list 
fips = df_sample['FIPS'].tolist()

In [0]:
# put our unemployed values into a list 
values = df_sample['Unemployment Rate (%)'].tolist()

# Now ready to plot!

In [13]:
fig = ff.create_choropleth(
    fips=fips,
    values=values,
    scope=['usa'],
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True,
    asp = 2.9, # aspectratio
    title_text = 'USA by Unemployment %',
    legend_title = '% unemployed'
)
fig.layout.template = None
fig.show()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




# World Choropleth

In [14]:
import plotly.express as px

gapminder = px.data.gapminder().query("year==2007")
fig = px.choropleth(gapminder, locations="iso_alpha",
                    color="lifeExp", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [0]:
import plotly.graph_objects as go

# Load data frame and tidy it.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

fig = go.Figure(data=go.Choropleth(
    locations=df['code'], # Spatial coordinates
    z = df['total exports'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [0]:
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Beef ' + df['beef'] + ' Dairy ' + df['dairy'] + '<br>' + \
    'Fruits ' + df['total fruits'] + ' Veggies ' + df['total veggies'] + '<br>' + \
    'Wheat ' + df['wheat'] + ' Corn ' + df['corn']

fig = go.Figure(data=go.Choropleth(
    locations=df['code'],
    z=df['total exports'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Millions USD"
))

fig.update_layout(
    title_text='2011 US Agriculture Exports by State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)


In [0]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

fig = go.Figure(data=go.Choropleth(
    locations = df['CODE'],
    z = df['GDP (BILLIONS)'],
    text = df['COUNTRY'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
))

fig.update_layout(
    title_text='2014 Global GDP',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)

fig.show()

In [0]:
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_ebola.csv')
df.head()

colors = ['rgb(239,243,255)', 'rgb(189,215,231)', 'rgb(107,174,214)', 'rgb(33,113,181)']
months = {6:'June', 7:'July', 8:'Aug', 9:'Sept'}

fig = go.Figure()

# scatter chart for outbreak size
for i in range(6,10)[::-1]:
    df_month = df.query('Month == %d' %i)
    fig.add_trace(go.Scattergeo(
        lon = df_month['Lon'],
        lat = df_month['Lat'],
        text = df_month['Value'],
        name = months[i],
        marker = dict(
            size=df_month['Value']/50,
            color=colors[i-6],
            line_width=0)
        )
    )

df_sept = df.query('Month == 9')
fig.data[0].update(text = df_sept['Value'].map('{:.0f}'.format).astype(str)+' '+\
                        df_sept['Country'],
                     mode = 'markers+text',
                     textposition = 'bottom center')


fig.add_trace(go.Choropleth(
        locationmode='country names',
        locations=df_sept['Country'],
        z=df_sept['Value'],
        text=df_sept['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    ))
fig.add_trace(go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['Africa'],
        mode = 'text',
        showlegend = False,
        geo = 'geo2'
    ))

fig.update_layout(
    title_text = 'Ebola cases reported by month in West Africa 2014<br> \
Source: <a href="https://data.hdx.rwlabs.org/dataset/rowca-ebola-cases">\
HDX</a>',
    geo = dict(
        resolution=50,
        scope='africa',
        showframe=False,
        showcoastlines=True,
        showland=True,
        landcolor="lightgray",
        countrycolor="white" ,
        coastlinecolor="white",
        projection_type='equirectangular',
        lonaxis_range=[ -15.0, -5.0],
        lataxis_range=[ 0.0, 12.0],
        domain = dict(x=[0, 1], y=[ 0, 1])
    ),
    geo2 = dict(
        scope='africa',
        showframe=False,
        showland=True,
        landcolor="lightgray",
        showcountries=False,
        domain=dict(x=[ 0, 0.6], y=[ 0, 0.6]),
        bgcolor='rgba(255, 255, 255, 0.0)',
    ),
    legend_traceorder = 'reversed'
)

fig.show()